# Day 1, Block B: Aggregations & GROUP BY

**Duration:** 25-30 minutes  
**Course:** ECBS5294 - Introduction to Data Science: Working with Data  
**Instructor:** Eduardo Ariño de la Rubia

---

## Learning Objectives

By the end of this session, you will be able to:

1. **Use aggregate functions** (COUNT, SUM, AVG, MIN, MAX)
2. **Group data** with GROUP BY
3. **Filter groups** with HAVING
4. **Understand the critical difference** between WHERE and HAVING
5. **Handle NULL values** in aggregations
6. **Write queries** that answer business questions with summaries

---

## 1. Setup & Recap

### What We Learned Last Time

In Notebook 1, we learned:
- **SELECT** - Choose columns
- **WHERE** - Filter rows
- **ORDER BY** - Sort results
- **NULL handling** - IS NULL, not = NULL!

### What's Next

Now we'll learn to **summarize** data:
- "How many transactions?"
- "What's the total revenue?"
- "What's the average price per category?"

This is where SQL gets really powerful for analytics!

---

In [1]:
# Imports
import duckdb
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

print("✅ Libraries imported!")

✅ Libraries imported!


In [2]:
# Connect to DuckDB and load cafe sales data with proper types
con = duckdb.connect(':memory:')

# Clean the data types (just like we did in Notebook 1!)
con.execute("""
    CREATE OR REPLACE TABLE cafe AS
    SELECT 
        "Transaction ID",
        Item,
        TRY_CAST(Quantity AS INTEGER) AS Quantity,
        TRY_CAST("Price Per Unit" AS DECIMAL(10,2)) AS "Price Per Unit",
        TRY_CAST("Total Spent" AS DECIMAL(10,2)) AS "Total Spent",
        "Payment Method",
        Location,
        TRY_CAST("Transaction Date" AS DATE) AS "Transaction Date"
    FROM '../data/day1/dirty_cafe_sales.csv'
""")

print("✅ Database connected and table loaded with clean types!")

✅ Database connected and table loaded with clean types!


In [3]:
# Quick check
con.execute("SELECT * FROM cafe LIMIT 3").df()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,NaN,Credit Card,In-store,2023-07-19


---

## 2. What Are Aggregate Functions?

> **Aggregate functions collapse many rows into one summary value**

Think about it:
- You have 10,000 transaction rows
- You want to know: "What's the total revenue?"
- Aggregate function: **Collapses 10,000 rows → 1 number**

### Common Aggregate Functions

| Function | What it does | Example |
|----------|--------------|----------|
| `COUNT(*)` | Counts all rows | How many transactions? |
| `COUNT(column)` | Counts non-NULL values | How many have payment method? |
| `SUM(column)` | Adds up numbers | Total revenue? |
| `AVG(column)` | Average (mean) | Average price? |
| `MIN(column)` | Smallest value | Cheapest item? |
| `MAX(column)` | Largest value | Most expensive item? |

---

## 3. COUNT: How Many Rows?

### COUNT(*) - Count All Rows

**Business question:** "How many transactions do we have?"

In [4]:
con.execute("""
    SELECT COUNT(*) AS total_transactions
    FROM cafe
""").df()

,total_transactions
0,10000


**What happened?**
- 10,000 rows → collapsed to 1 row
- Result: The count

This is the essence of aggregation!

### COUNT(column) - Count Non-NULL Values

**Business question:** "How many transactions have a recorded payment method?"

In [5]:
con.execute("""
    SELECT 
        COUNT(*) AS all_rows,
        COUNT("Payment Method") AS with_payment_method
    FROM cafe
""").df()

,all_rows,with_payment_method
0,10000,7421


**Critical difference:**
- `COUNT(*)` counts **every row**, including rows with NULLs
- `COUNT(column)` counts only **non-NULL values** in that column

This is one of the most important NULL behaviors to understand!

### Example: Finding Missing Data Percentage

In [6]:
con.execute("""
    SELECT 
        COUNT(*) AS total_rows,
        COUNT("Payment Method") AS with_payment,
        COUNT(*) - COUNT("Payment Method") AS missing_payment,
        ROUND(100.0 * (COUNT(*) - COUNT("Payment Method")) / COUNT(*), 2) AS pct_missing
    FROM cafe
""").df()

,total_rows,with_payment,missing_payment,pct_missing
0,10000,7421,2579,25.79


**Business insight:** ~26% of our transactions are missing payment method data!

---

## 4. SUM: Add It Up

**Business question:** "What's our total revenue?"

In [7]:
con.execute("""
    SELECT 
        SUM("Price Per Unit"* Quantity) AS total_revenue
    FROM cafe
    WHERE "Price Per Unit"IS NOT NULL AND Quantity IS NOT NULL
""").df()

,total_revenue
0,80478.5


**What happened:** Calculated `Price * Quantity` for each row, then summed all values.

**Note:** We filter out NULLs in WHERE because we can't calculate revenue if price or quantity is NULL.

### Example: Multiple Sums

In [8]:
con.execute("""
    SELECT 
        SUM("Price Per Unit"* Quantity) AS total_revenue,
        SUM(Quantity) AS total_items_sold
    FROM cafe
    WHERE "Price Per Unit"IS NOT NULL AND Quantity IS NOT NULL
""").df()

,total_revenue,total_items_sold
0,80478.5,27230.0


---

## 5. AVG, MIN, MAX

**Business questions:**
- "What's the average price per unit?"
- "What's our cheapest item?"
- "What's our most expensive item?"

In [9]:
con.execute("""
    SELECT 
        ROUND(AVG("Price Per Unit"), 2) AS avg_price,
        MIN("Price Per Unit") AS min_price,
        MAX("Price Per Unit") AS max_price
    FROM cafe
    WHERE "Price Per Unit"IS NOT NULL
""").df()

,avg_price,min_price,max_price
0,2.95,1.0,5.0


### All Together: Complete Summary Stats

In [10]:
con.execute("""
    SELECT 
        COUNT(*) AS transactions_with_data,
        SUM("Price Per Unit" * Quantity) AS total_revenue,
        ROUND(AVG("Price Per Unit" * Quantity), 2) AS avg_transaction_value,
        MIN("Price Per Unit" * Quantity) AS min_transaction,
        MAX("Price Per Unit" * Quantity) AS max_transaction
    FROM cafe
    WHERE "Price Per Unit" IS NOT NULL AND Quantity IS NOT NULL
""").df()

,transactions_with_data,total_revenue,avg_transaction_value,min_transaction,max_transaction
0,9006,80478.5,8.94,1.0,25.0


**This is powerful!** One query gives you a complete statistical summary.

---

## 6. NULL Behavior in Aggregates

> **🚨 CRITICAL: Understand how aggregates handle NULL**

Let's see it in action:

In [11]:
# Create a small example table with NULLs
con.execute("""
    CREATE OR REPLACE TABLE example AS
    SELECT * FROM (VALUES 
        (1, 10),
        (2, 20),
        (3, NULL),
        (4, 30)
    ) AS t(id, value)
""")

con.execute("SELECT * FROM example").df()

,id,value
0,1,10
1,2,20
2,3,<NA>
3,4,30


In [12]:
# How do aggregates handle the NULL?
con.execute("""
    SELECT 
        COUNT(*) AS count_all_rows,
        COUNT(value) AS count_non_null_values,
        SUM(value) AS sum_values,
        AVG(value) AS avg_values
    FROM example
""").df()

,count_all_rows,count_non_null_values,sum_values,avg_values
0,4,3,60.0,20.0


**What happened?**
- `COUNT(*)` = 4 (all rows, including the one with NULL)
- `COUNT(value)` = 3 (only non-NULL values)
- `SUM(value)` = 60 (10 + 20 + 30, NULL ignored)
- `AVG(value)` = 20 (sum of 60 ÷ 3 non-NULL values, **not** ÷ 4)

**Key insight:** Most aggregates **ignore NULLs** (except COUNT(*)).

### Why This Matters

**Scenario:** You have revenue data with some missing values.

❌ **If you use 0 for missing:**
```python
df['revenue'].fillna(0)
```
→ Average includes the zeros (drags average down)

✅ **If you use NULL for missing:**
```python
# Leave as NULL
```
→ Average excludes NULLs (average of known values)

**Different meaning, different results!** This is what we learned in Block A about missing values.

---

## 7. GROUP BY: The Game Changer

> **Mental model: "For each X, calculate Y"**

So far, we've aggregated **all rows**:
- "What's the **total** revenue?" → One number

But what if we want:
- "What's the total revenue **per payment method**?" → One number per payment method
- "What's the total revenue **per item**?" → One number per item

This is what **GROUP BY** does!

### Example 1: Count by Payment Method

**Business question:** "How many transactions per payment method?"

In [13]:
con.execute("""
    SELECT 
        "Payment Method",
        COUNT(*) AS transaction_count
    FROM cafe
    GROUP BY "Payment Method"
    ORDER BY transaction_count DESC
""").df()

,Payment Method,transaction_count
0,None,2579
1,Digital Wallet,2291
2,Credit Card,2273
3,Cash,2258
4,ERROR,306
5,UNKNOWN,293


**What happened?**

1. GROUP BY split data into groups (one per payment method)
2. COUNT(*) calculated within each group
3. Result: One row per payment method

**Before:** 10,000 rows  
**After:** 3-5 rows (one per payment method)

### Visualizing GROUP BY

```
Original data (1000s of rows):
Payment_Method | Price
Credit Card    | 5.00
Credit Card    | 3.50
Cash           | 2.00
Cash           | 4.00
Credit Card    | 6.00
...

GROUP BY Payment_Method:

Group 1 (Credit Card): 5.00, 3.50, 6.00, ...
Group 2 (Cash): 2.00, 4.00, ...
Group 3 (Debit): ...

Then COUNT(*) within each group:

Payment_Method | transaction_count
Credit Card    | 3452
Cash           | 2103
Debit          | 1876
```

### Example 2: Revenue by Item

**Business question:** "What's our revenue per product?"

In [14]:
con.execute("""
    SELECT 
        Item,
        SUM("Price Per Unit"* Quantity) AS total_revenue
    FROM cafe
    WHERE "Price Per Unit"IS NOT NULL AND Quantity IS NOT NULL
    GROUP BY Item
    ORDER BY total_revenue DESC
    LIMIT 10
""").df()

,Item,total_revenue
0,Salad,15600.0
1,Sandwich,12296.0
2,Smoothie,12132.0
3,Juice,9561.0
4,Cake,9540.0
5,Coffee,6424.0
6,Tea,4431.0
7,Cookie,2898.0
8,None,2651.0
9,UNKNOWN,2550.0


**Business insight:** These are our top-selling products by revenue!

### Example 3: Multiple Aggregates per Group

**Business question:** "For each item, show transactions, revenue, and average price."

In [15]:
con.execute("""
    SELECT 
        Item,
        COUNT(*) AS transactions,
        ROUND(SUM("Price Per Unit"* Quantity), 2) AS total_revenue,
        ROUND(AVG("Price Per Unit"), 2) AS avg_price
    FROM cafe
    WHERE "Price Per Unit"IS NOT NULL AND Quantity IS NOT NULL
    GROUP BY Item
    ORDER BY total_revenue DESC
    LIMIT 10
""").df()

,Item,transactions,total_revenue,avg_price
0,Salad,1030,15600.0,5.00
1,Sandwich,1022,12296.0,4.00
2,Smoothie,994,12132.0,4.00
3,Juice,1063,9561.0,3.00
4,Cake,1042,9540.0,3.00
5,Coffee,1048,6424.0,2.00
6,Tea,972,4431.0,1.50
7,Cookie,976,2898.0,1.00
8,None,293,2651.0,2.98
9,UNKNOWN,304,2550.0,2.88


**Rich analysis!** Multiple metrics per group.

---

### ⏸️ Pause and Try!

**Your task:** Write a complete GROUP BY query to analyze revenue by payment method.

**Requirements:**
1. Calculate **total revenue** per payment method (`SUM` of Price × Quantity)
2. Calculate **average transaction value** per payment method (`AVG` of Price × Quantity)
3. Include **count of transactions** per payment method
4. Filter out NULL prices and quantities (use `WHERE`)
5. Order results by **total revenue descending**

Replace the placeholder query in the cell below with your complete SQL query.

---

### ⏸️ Pause and Try!

**Your task:** Write a complete GROUP BY query to analyze revenue by payment method.

**Requirements:**
1. Calculate **total revenue** per payment method (`SUM` of Price × Quantity)
2. Calculate **average transaction value** per payment method (`AVG` of Price × Quantity)
3. Include **count of transactions** per payment method
4. Filter out NULL prices and quantities (use `WHERE`)
5. Order results by **total revenue descending**

Replace the placeholder query in the cell below with your complete SQL query.

### Example 4: GROUP BY Multiple Columns

**Business question:** "Revenue by item AND payment method."

In [16]:
con.execute("""
    SELECT 
        Item,
        "Payment Method",
        COUNT(*) AS transactions,
        ROUND(SUM("Price Per Unit"* Quantity), 2) AS revenue
    FROM cafe
    WHERE "Price Per Unit"IS NOT NULL 
      AND Quantity IS NOT NULL
      AND "Payment Method" IS NOT NULL
    GROUP BY Item, "Payment Method"
    ORDER BY revenue DESC
    LIMIT 15
""").df()

,Item,Payment Method,transactions,revenue
0,Salad,Cash,260,3890.0
1,Salad,Credit Card,256,3765.0
2,Salad,Digital Wallet,217,3360.0
3,Sandwich,Digital Wallet,231,2884.0
4,Smoothie,Digital Wallet,233,2824.0
5,Smoothie,Cash,219,2800.0
6,Sandwich,Cash,225,2756.0
7,Smoothie,Credit Card,223,2724.0
8,Sandwich,Credit Card,231,2712.0
9,Cake,Cash,254,2307.0


**What happened:** One row for each **combination** of (Item, Payment_Method).

**Insight:** "Coffee paid by Credit Card" vs "Coffee paid by Cash" are separate groups.

---

## 8. GROUP BY Rules

> **🚨 RULE: Every column in SELECT must be either:**
> 1. **In the GROUP BY clause**, OR
> 2. **Inside an aggregate function**

Let's see what happens if you break this rule:

In [17]:
# ❌ WRONG - "Price Per Unit" is not in GROUP BY and not aggregated
try:
    con.execute("""
        SELECT 
            Item,
            "Price Per Unit",
            COUNT(*) AS transactions
        FROM cafe
        GROUP BY Item
    """).df()
except Exception as e:
    print(f"❌ Error: {e}")

❌ Error: Binder Error: column "Price Per Unit" must appear in the GROUP BY clause or must be part of an aggregate function.
Either add it to the GROUP BY list, or use "ANY_VALUE(Price Per Unit)" if the exact value of "Price Per Unit" is not important.

LINE 4:             "Price Per Unit",
                    ^


**Why this fails:** 
- We're grouping by `Item`
- But `Price` is not in GROUP BY
- Each item has MANY prices - which one should SQL show?
- SQL doesn't know, so it errors!

In [18]:
# ✅ CORRECT - "Price Per Unit" is aggregated
con.execute("""
    SELECT 
        Item,
        ROUND(AVG("Price Per Unit"), 2) AS avg_price,
        COUNT(*) AS transactions
    FROM cafe
    WHERE "Price Per Unit"IS NOT NULL
    GROUP BY Item
    ORDER BY transactions DESC
    LIMIT 10
""").df()

,Item,avg_price,transactions
0,Juice,3.00,1110
1,Coffee,2.00,1108
2,Cake,3.00,1085
3,Sandwich,4.00,1082
4,Salad,5.00,1082
5,Smoothie,4.00,1036
6,Cookie,1.00,1026
7,Tea,1.50,1023
8,UNKNOWN,2.88,326
9,None,2.98,310


**Now it works!** Because `Price` is aggregated with `AVG()`.

---

## 9. WHERE vs HAVING: The Critical Distinction

> **🚨 MOST IMPORTANT CONCEPT IN THIS NOTEBOOK**

### The Difference

| | WHERE | HAVING |
|---|---|---|
| **What it filters** | Individual rows | Groups (after aggregation) |
| **When it runs** | BEFORE grouping | AFTER grouping |
| **Can use aggregates?** | ❌ No | ✅ Yes |
| **Example** | "Transactions over $5" | "Items with >100 transactions" |

### Example 1: WHERE Filters Rows BEFORE Grouping

**Business question:** "Revenue per item, but only include transactions $4 or more."

In [19]:
con.execute("""
    SELECT 
        Item,
        COUNT(*) AS transactions,
        ROUND(SUM("Price Per Unit" * Quantity), 2) AS revenue
    FROM cafe
    WHERE "Price Per Unit" >= 4.00  -- Filter ROWS before grouping
      AND "Price Per Unit" IS NOT NULL 
      AND Quantity IS NOT NULL
    GROUP BY Item
    ORDER BY revenue DESC
    LIMIT 10
""").df()

,Item,transactions,revenue
0,Salad,1030,15600.0
1,Sandwich,1022,12296.0
2,Smoothie,994,12132.0
3,None,113,1512.0
4,UNKNOWN,106,1382.0
5,ERROR,94,1272.0


**Execution order:**
1. WHERE filters out rows with Price < $4
2. Then GROUP BY groups remaining rows
3. Then COUNT and SUM within each group

### Example 2: HAVING Filters Groups AFTER Aggregation

**Business question:** "Show me items with more than 100 transactions."

In [20]:
con.execute("""
    SELECT 
        Item,
        COUNT(*) AS transactions,
        ROUND(SUM("Price Per Unit"* Quantity), 2) AS revenue
    FROM cafe
    WHERE "Price Per Unit"IS NOT NULL AND Quantity IS NOT NULL
    GROUP BY Item
    HAVING COUNT(*) > 100  -- Filter GROUPS after aggregation
    ORDER BY transactions DESC
""").df()

,Item,transactions,revenue
0,Juice,1063,9561.0
1,Coffee,1048,6424.0
2,Cake,1042,9540.0
3,Salad,1030,15600.0
4,Sandwich,1022,12296.0
5,Smoothie,994,12132.0
6,Cookie,976,2898.0
7,Tea,972,4431.0
8,UNKNOWN,304,2550.0
9,None,293,2651.0


**Execution order:**
1. WHERE filters individual rows (NULL check)
2. GROUP BY groups remaining rows
3. COUNT and SUM within each group
4. HAVING filters out groups with COUNT ≤ 100

### Example 3: Using Both WHERE and HAVING

**Business question:** "Items with >50 transactions, but only count transactions over $3."

In [21]:
con.execute("""
    SELECT 
        Item,
        COUNT(*) AS transactions,
        ROUND(AVG("Price Per Unit"), 2) AS avg_price,
        ROUND(SUM("Price Per Unit"* Quantity), 2) AS revenue
    FROM cafe
    WHERE "Price Per Unit"> 3.00          -- Filter rows: only include price > $3
      AND "Price Per Unit"IS NOT NULL 
      AND Quantity IS NOT NULL
    GROUP BY Item
    HAVING COUNT(*) > 50        -- Filter groups: only show items with >50 transactions
    ORDER BY revenue DESC
    LIMIT 10
""").df()

,Item,transactions,avg_price,revenue
0,Salad,1030,5.00,15600.0
1,Sandwich,1022,4.00,12296.0
2,Smoothie,994,4.00,12132.0
3,None,113,4.32,1512.0
4,UNKNOWN,106,4.41,1382.0
5,ERROR,94,4.40,1272.0


**Execution order:**
1. **WHERE** filters rows (Price > $3)
2. **GROUP BY** groups remaining rows by Item
3. Aggregates calculated (COUNT, AVG, SUM)
4. **HAVING** filters groups (COUNT > 50)
5. **ORDER BY** sorts results

### Side-by-Side Comparison

Let's see the difference clearly:

In [22]:
# Query 1: WHERE only
result1 = con.execute("""
    SELECT 
        "Payment Method",
        COUNT(*) AS transactions
    FROM cafe
    WHERE "Price Per Unit" > 3.00  -- Filter rows
      AND "Price Per Unit" IS NOT NULL
    GROUP BY "Payment Method"
    ORDER BY transactions DESC
""").df()

print('Query 1: WHERE "Price Per Unit" > 3 (filters rows before grouping)')
print(result1)
print()

Query 1: WHERE "Price Per Unit" > 3 (filters rows before grouping)
   Payment Method  transactions
0            None           885
1     Credit Card           831
2            Cash           813
3  Digital Wallet           802
4           ERROR           106
5         UNKNOWN            98



In [23]:
# Query 2: HAVING only
result2 = con.execute("""
    SELECT 
        "Payment Method",
        COUNT(*) AS transactions
    FROM cafe
    WHERE "Price Per Unit"IS NOT NULL
    GROUP BY "Payment Method"
    HAVING COUNT(*) > 100  -- Filter groups
""").df()

print("Query 2: HAVING COUNT > 100 (filters groups after aggregation)")
print(result2)

Query 2: HAVING COUNT > 100 (filters groups after aggregation)
   Payment Method  transactions
0            Cash          2143
1     Credit Card          2145
2            None          2439
3         UNKNOWN           279
4           ERROR           286
5  Digital Wallet          2175


**See the difference?**
- Query 1: Fewer transactions per group (because we filtered rows first)
- Query 2: Some payment methods excluded entirely (because their counts were ≤100)

### Common Mistake: Using Aggregate in WHERE

❌ **This doesn't work:**

In [24]:
# ❌ WRONG - Can't use aggregate in WHERE
try:
    con.execute("""
        SELECT 
            Item,
            COUNT(*) AS transactions
        FROM cafe
        WHERE COUNT(*) > 100  -- ❌ Wrong!
        GROUP BY Item
    """).df()
except Exception as e:
    print(f"❌ Error: {str(e)[:100]}...")

❌ Error: Binder Error: WHERE clause cannot contain aggregates!

LINE 6:         WHERE COUNT(*) > 100  -- ❌ Wr...


✅ **Use HAVING instead:**

---

### ⏸️ Pause and Try!

**Your task:** Write a complete query using HAVING to filter aggregated groups.

**Requirements:**
1. Find items with **more than 200 transactions**
2. Show: Item name, transaction count, and total revenue
3. Filter out NULL prices and quantities (use `WHERE`)
4. Use `HAVING` to filter groups (NOT WHERE - this filters after aggregation)
5. Order results by **revenue descending**

Replace the placeholder query in the cell below with your complete SQL query.

---

### ⏸️ Pause and Try!

**Your task:** Write a complete query using HAVING to filter aggregated groups.

**Requirements:**
1. Find items with **more than 200 transactions**
2. Show: Item name, transaction count, and total revenue
3. Filter out NULL prices and quantities (use `WHERE`)
4. Use `HAVING` to filter groups (NOT WHERE - this filters after aggregation)
5. Order results by **revenue descending**

Replace the placeholder query in the cell below with your complete SQL query.

In [25]:
# ✅ CORRECT - Use HAVING for aggregates
con.execute("""
    SELECT 
        Item,
        COUNT(*) AS transactions
    FROM cafe
    GROUP BY Item
    HAVING COUNT(*) > 100  -- ✅ Correct!
    ORDER BY transactions DESC
    LIMIT 10
""").df()

,Item,transactions
0,Juice,1171
1,Coffee,1165
2,Salad,1148
3,Cake,1139
4,Sandwich,1131
5,Smoothie,1096
6,Cookie,1092
7,Tea,1089
8,UNKNOWN,344
9,None,333


---

## 10. Decision Guide: WHERE vs HAVING

**Ask yourself: "Am I filtering individual rows or aggregated groups?"**

### Use WHERE when:
- ✅ Filtering **individual rows**
- ✅ Condition doesn't involve aggregates
- ✅ Examples:
  - "Only transactions over $5"
  - "Only coffee sales"
  - "Only credit card payments"

### Use HAVING when:
- ✅ Filtering **groups** after aggregation
- ✅ Condition involves aggregates (COUNT, SUM, AVG, etc.)
- ✅ Examples:
  - "Items with more than 100 transactions"
  - "Categories with revenue over $1000"
  - "Customers with average purchase over $50"

### Use BOTH when:
```sql
SELECT category, COUNT(*), SUM(revenue)
FROM sales
WHERE price > 10         -- Filter rows first
GROUP BY category
HAVING COUNT(*) > 100    -- Then filter groups
```

---

## 11. Putting It All Together

### Complex Business Question

**Question:** 
> "Which items had more than 50 transactions with prices between $4-$7, and what was their total revenue? Show top 10 by revenue."

In [26]:
con.execute("""
    SELECT 
        Item,
        COUNT(*) AS transactions,
        ROUND(AVG("Price Per Unit"), 2) AS avg_price,
        ROUND(SUM("Price Per Unit"* Quantity), 2) AS total_revenue
    FROM cafe
    WHERE "Price Per Unit" BETWEEN 4.00 AND 7.00  -- Filter rows: price range
      AND "Price Per Unit"IS NOT NULL 
      AND Quantity IS NOT NULL
    GROUP BY Item                      -- Group by item
    HAVING COUNT(*) > 50               -- Filter groups: >50 transactions
    ORDER BY total_revenue DESC        -- Sort by revenue
    LIMIT 10                           -- Top 10
""").df()

,Item,transactions,avg_price,total_revenue
0,Salad,1030,5.00,15600.0
1,Sandwich,1022,4.00,12296.0
2,Smoothie,994,4.00,12132.0
3,None,113,4.32,1512.0
4,UNKNOWN,106,4.41,1382.0
5,ERROR,94,4.40,1272.0


**Breaking it down:**
1. **WHERE** - Include only rows with price $4-$7
2. **GROUP BY** - Group by Item
3. **Aggregates** - COUNT, AVG, SUM within each group
4. **HAVING** - Keep only items with >50 transactions
5. **ORDER BY** - Sort by revenue descending
6. **LIMIT** - Show top 10

This query answers a complex business question in 10 lines!

---

## 12. Common Mistakes Summary

### ❌ Mistake 1: Using aggregate in WHERE
```sql
WHERE COUNT(*) > 100  -- ❌ Wrong!
HAVING COUNT(*) > 100 -- ✅ Correct
```

### ❌ Mistake 2: Forgetting column in GROUP BY
```sql
SELECT Item, Price, COUNT(*)  -- ❌ Price not in GROUP BY or aggregated
FROM sales
GROUP BY Item
```

### ❌ Mistake 3: Expecting detail rows after GROUP BY
```sql
-- After GROUP BY, you only get one row per group
-- Detail rows are gone!
SELECT Item, COUNT(*)   -- Only 10 rows if 10 unique items
FROM sales              -- Even if sales has 10,000 rows
GROUP BY Item
```

### ❌ Mistake 4: Not handling NULLs
```sql
-- NULLs will form their own group!
SELECT Payment_Method, COUNT(*)
FROM sales
GROUP BY Payment_Method  -- Includes a NULL group if any NULLs exist
```

---

## 13. Summary: What We Learned

### Aggregate Functions
1. **COUNT(*)** - Counts all rows (including NULLs)
2. **COUNT(column)** - Counts non-NULL values
3. **SUM, AVG, MIN, MAX** - All ignore NULLs

### GROUP BY
4. **Mental model:** "For each X, calculate Y"
5. **Rule:** SELECT columns must be in GROUP BY or aggregated
6. **Result:** One row per group (collapses detail rows)

### WHERE vs HAVING
7. **WHERE** - Filters rows BEFORE grouping
8. **HAVING** - Filters groups AFTER aggregation
9. **Can't** use aggregates in WHERE
10. **Must** use aggregates in HAVING (that's its purpose!)

### Most Important Takeaway
> **🚨 Filtering rows? → WHERE**  
> **🚨 Filtering groups? → HAVING**

Master this distinction and you've mastered aggregations!

---

## Next: Window Functions

In the next notebook, we'll learn **window functions** - GROUP BY's more flexible cousin:
- "I want the total per category, but keep all detail rows"
- "Show me the latest transaction per customer"
- "Calculate month-over-month growth"

**The key difference:**
- GROUP BY: Collapses rows → fewer rows
- Window functions: Keep all rows, add calculations

This is advanced SQL, but incredibly powerful!

**Ready?** Open `day1_block_b_03_window_functions.ipynb`

---

**Excellent progress!** 🎉 You can now summarize data like a pro.

---

## ⏸️ Take a 5-Minute Break!

Before starting **Window Functions** (Notebook 3), give your brain a rest.

Window functions are **conceptually challenging** - they require a different mental model than GROUP BY. You'll learn better with a fresh mind.

☕ **Stretch, get water, clear your head!**

**When you're ready:** Open `day1_block_b_03_window_functions.ipynb`

> **Remember:** Windows PRESERVE rows, GROUP BY COLLAPSES rows. This is the key insight you'll need next!